In [95]:
import numpy as np
import pandas as pd
import os
import vader_feature
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [96]:
analyzer = SentimentIntensityAnalyzer()
special_words = vader_feature.new_words
analyzer.lexicon.update(special_words)
path = '../words/pos-neg'
files = os.listdir(path)
files.sort(key = lambda x: int(x[0]))


In [98]:
from collections import defaultdict
import re 
dict_list = []
for f_name in files:
    count_dict = defaultdict(int)
    fpath = os.path.join(path, f_name)
    with open(fpath,encoding='utf8',errors='ignore') as f:
        content = f.read()
        content = re.sub("[\[,']", '', content)
    for i,w in enumerate(content.split()):
        if w.isalpha(): #remove broken letters
            count_dict[w] = i
    dict_list.append(count_dict)

In [99]:
df = pd.DataFrame()
for i,d in enumerate(dict_list):
    col_name = f"{-1 + (i * 0.2):.2f}"
    add_col = pd.Series(d,dtype = np.int32, name = col_name)
    df = pd.concat([df,add_col],axis=1)

In [100]:
index = np.array(df.index)
df['vader'] = df.index
df['vader'] = df['vader'].apply(analyzer.polarity_scores)
df['vader'] = df['vader'].apply(lambda x: x['compound'])

In [149]:
df.to_csv('raw.csv')

#### 가중치가 0인 단어 구분하기
* Vader 결과 값은 0이지만, 감정정보가 포함돼 있어 수정해줘야 하는 단어들이 존재한다.
* moon,hodl,ATH,buy the dip 등의 meme관련 용어나 특수한 상황에서 쓰이는 단어들이 바로 해당 단어들이다. 
* 빈도수를 기준으로 순위를 매겨놓은 df에서 위와 같이 따로 가중치를 부여해야할 단어가 있는지 확인해보자.

In [145]:
df_zero = df[df['vader'] == 0] #vader가중치가 0인 단어들
na_count = df_zero.isna().sum(axis=1)
df_zero = df_zero[na_count > 5] #5개 이상의 구간에서 전부 150위권 안의 빈도수에 존재한다면 무시한다.

모든 구간에서 많이 쓰이는 단어는 따로 가중치를 부여할 필요 없다. 
- 이는 통상적으로 쓰이는 단어일 가능성이 높기 때문이다. 
- 예를 들자면 BTC,Crypto,Money 등이 있다.

In [133]:
df_zero_pos = df_zero.loc[::,'0.40':] #긍정적인 구간에서의 df를 생성한다.
df_zero_pos

,0.40,0.60,0.80,vader
history,NaN,NaN,NaN,0.0
youtube,NaN,NaN,NaN,0.0
else,NaN,NaN,NaN,0.0
many,NaN,NaN,128.0,0.0
posts,NaN,NaN,NaN,0.0
...,...,...,...,...
community,NaN,NaN,108.0,0.0
find,NaN,NaN,124.0,0.0
found,NaN,NaN,132.0,0.0
making,NaN,NaN,135.0,0.0


In [141]:
df_zero_pos = df_zero_pos[df_zero_pos.isna().sum(axis=1) <= 1] # 결측치가 1개 이하로 존재하는 단어만 고른다.
df_zero_pos.sort_values(by='0.80')

,0.40,0.60,0.80
link,NaN,87.0,29.0
card,94.0,48.0,41.0
dip,NaN,58.0,58.0
everyone,NaN,137.0,64.0
platform,NaN,89.0,66.0
trade,NaN,84.0,72.0
browser,NaN,133.0,82.0
run,NaN,82.0,98.0
video,NaN,115.0,114.0
check,NaN,135.0,117.0


In [142]:
df_zero_pos.to_csv('pos.csv')

In [146]:
df_zero_neg = df_zero.loc[::,:'-0.60']
df_zero_neg = df_zero_neg[df_zero_neg.isna().sum(axis=1) <= 1] # 결측치가 1개 이하로 존재하는 단어만 고른다.

In [147]:
df_zero_neg

,-1.00,-0.80,-0.60
youtube,90.0,50.0,44.0
else,96.0,NaN,132.0
many,100.0,NaN,125.0
financial,110.0,49.0,139.0
live,128.0,107.0,NaN
say,142.0,120.0,143.0
big,NaN,63.0,70.0
trying,NaN,93.0,131.0
funds,NaN,100.0,108.0
binance,NaN,106.0,104.0


In [148]:
df_zero_neg.to_csv('neg.csv')